In [4]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape
from dash_extensions.javascript import assign
from dash import Output, Input
from datetime import datetime, date
import math

Useful Functions!

In [5]:
# Utility Function to Read API Responses

# read_response checks if the response of the server is valid JSON

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Risposta non valida JSON!")
        print("Contenuto della risposta:", t.text)
        data = None 

# get_measurement_unit returns the measurement unit of a given pollutant

def get_measurement_unit(pollutant):
    t=requests.post(url="http://127.0.0.1:5000/api/units", json={"var_pollutant": pollutant}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data[0]

# all_pollutant_dropdown populates the dropdown with all the pollutants, even those that have no recorded values

def all_pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/all_pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list
    
# Get List of Available Provinces

def get_provinces_list():
    response = requests.get("http://127.0.0.1:5000/api/provinces")
    return read_response(response) or []

# Get List of Available Pollutants

def get_pollutants_list():
    response = requests.get("http://127.0.0.1:5000/api/pollutants")
    return read_response(response) or []


Definition of variables for visualization

In [6]:

############################################# EU_DV_2 #############################################

app = Dash(__name__)

app.layout = html.Div([
    html.H1('Bugs_project: Air quality analysis'),
    html.P('Description of the dashboard functionalities'),

    html.Div([
        html.H2('EU_DV_2 Threshold definition – time-series'),
        html.P("Select a pollutant, province, date range, and threshold to visualize which days exceeded it."),

        dcc.Dropdown(
            id='all-pollutant-dropdown_EU_DV_2',
            options=get_pollutants_list(),
            placeholder="Select a pollutant",
            style={'margin-top': 10}
        ),

        dcc.Dropdown(
            id='province-dropdown_EU_DV_2',
            options=get_provinces_list(),
            placeholder="Select a province",
            style={'margin-top': 10}
        ),

        dcc.DatePickerRange(
            id='date-range-picker_EU_DV_2',
            min_date_allowed=date(2000, 1, 1),
            max_date_allowed=date(2025, 12, 31),
            start_date_placeholder_text="Start Date",
            end_date_placeholder_text="End Date",
            display_format='YYYY-MM-DD',
            style={'margin-top': 10}
        ),

        html.Div([
            html.Label("Threshold:"),
            html.Span(id='unit-label_EU_DV_2', style={'margin-left': '10px', 'font-weight': 'bold'}),
        dcc.Slider(
            id='threshold-slider_EU_DV_2',
            min=0,
            max=80,
            step=1,
            value=50,
            marks={i: str(i) for i in range(0, 201, 20)},
            tooltip={"placement": "bottom", "always_visible": True},
            included=False,
            updatemode='drag'
        )
        ], style={'margin-top': '15px', 'margin-bottom': '30px'}),


        html.Div([
            html.Button(
                'Visualize threshold exceedance',
                id='button_EU_DV_2',
                n_clicks=0,
                style={'margin-top': 20, 'margin-bottom': 20}
            )
        ], style={'textAlign': 'center'}),

        html.Div(id='exceedance-percent_EU_DV_2', style={'textAlign': 'center'}),
        dcc.Graph(id='histogram-output_EU_DV_2')
    ])
])

############################################# Callback EU_DV_2 ###################################################################

@app.callback(
    Output('histogram-output_EU_DV_2', 'figure'),
    Output('exceedance-percent_EU_DV_2', 'children'),
    Input('button_EU_DV_2', 'n_clicks'),
    State('all-pollutant-dropdown_EU_DV_2', 'value'),
    State('province-dropdown_EU_DV_2', 'value'),
    State('date-range-picker_EU_DV_2', 'start_date'),
    State('date-range-picker_EU_DV_2', 'end_date'),
    State('threshold-slider_EU_DV_2', 'value')
)
def update_time_series(n_clicks, pollutant, province, start_date, end_date, threshold):
    if not all([pollutant, province, start_date, end_date, threshold]):
        return {}, "Please select all inputs."

    payload = {
        "var_pollutant": pollutant,
        "var_start_date": f"{start_date} 00:00:00",
        "var_end_date":   f"{end_date} 23:59:59",
        "var_province":   province,
        "var_threshold":  -9999          # ↩ chiediamo tutti i dati
    }

    try:
        r = requests.post("http://127.0.0.1:5000/api/EU_DV_2", json=payload, timeout=10)
        r.raise_for_status()
        data = r.json()
        if not data:
            return {}, "No data found for the selected filters."

        # -------------------- prepara i dati --------------------
        df = pd.DataFrame(data)
        df['data'] = pd.to_datetime(df['data'])
        per_day = (
            df.groupby(df['data'].dt.date)['valore'].max()
              .reset_index()
              .rename(columns={'valore': 'Max value', 'data': 'Timeframe'})
        )
        per_day['Exceed'] = per_day['Max value'] > threshold

        # -------------------- percentuale superamento -----------
        tot_days   = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days + 1
        exceed_cnt = per_day['Exceed'].sum()
        percent    = exceed_cnt / tot_days * 100

        # -------------------- grafico ---------------------------
        fig = px.bar(
            per_day,
            x='Timeframe',
            y='Max value',
            color='Exceed',
            color_discrete_map={True: 'firebrick', False: 'lightgrey'},
            title="Daily max concentration"
        )
        fig.add_hline(
            y=threshold,
            line_dash="dash",
            line_color="blue",
            annotation_text=f"Threshold = {threshold}"
        )
        fig.update_layout(legend_title_text="Exceed")

        # -------------------- formato asse X --------------------
        num_days = len(per_day)
        # mostro ogni 1 → 2 → 4 giorni a seconda di quanto è lungo il periodo
        step_days = 1 if num_days <= 10 else 2 if num_days <= 30 else 4

        fig.update_xaxes(
            tickformat="%Y-%m-%d",        # solo data
            tickangle=45,                 # etichette inclinate
            tickmode="linear",
            dtick=step_days * 86_400_000  # millisecondi per step
        )

        msg = f"Exceeded on {exceed_cnt} / {tot_days} days ({percent:.1f} %)"
        return fig, msg

    except Exception as e:
        return {}, f"Error: {e}"

@app.callback(
    Output('date-range-picker_EU_DV_2', 'min_date_allowed'),
    Output('date-range-picker_EU_DV_2', 'max_date_allowed'),
    Input('all-pollutant-dropdown_EU_DV_2', 'value')
)
def update_date_range_limits(pollutant):
    return fetch_pollutant_date_range(pollutant)


def fetch_pollutant_date_range(pollutant):
    # se il dropdown è ancora vuoto → ritorna range di default
    if not pollutant:
        # (opzionale) log di debug
        # print("fetch_pollutant_date_range: pollutant None → uso default")
        return date(2000, 1, 1), date(2025, 12, 31)

    try:
        r = requests.post(
            "http://127.0.0.1:5000/api/EU_DV_2/date_range",
            json={"var_pollutant": pollutant},
            timeout=5
        )
        r.raise_for_status()
        data = r.json()

        # se l’API non restituisce date valide
        if not data.get("start_date") or not data.get("end_date"):
            print(f"Nessuna data trovata per {pollutant}")
            return date(2000, 1, 1), date(2025, 12, 31)

        # converte ISO con orario → date pure
        min_d = datetime.fromisoformat(data["start_date"]).date()
        max_d = datetime.fromisoformat(data["end_date"]).date()
        return min_d, max_d

    except Exception as e:
        print("Errore fetch_pollutant_date_range:", e)
        return date(2000, 1, 1), date(2025, 12, 31)

@app.callback(
    Output('unit-label_EU_DV_2', 'children'),
    Input('all-pollutant-dropdown_EU_DV_2', 'value')
)
def update_unit_label(pollutant):
    if not pollutant:
        return ""
    try:
        unit = get_measurement_unit(pollutant)
        return f"[Unit: {unit}]"
    except Exception as e:
        print("Errore get_measurement_unit:", e)
        return "[Unit: N/A]"

@app.callback(
    Output('threshold-slider_EU_DV_2', 'min'),
    Output('threshold-slider_EU_DV_2', 'max'),
    Output('threshold-slider_EU_DV_2', 'step'),
    Output('threshold-slider_EU_DV_2', 'value'),
    Output('threshold-slider_EU_DV_2', 'marks'),
    Input('all-pollutant-dropdown_EU_DV_2', 'value')
)
def update_slider_limits(pollutant):
    try:
        if not pollutant:
            raise ValueError("No pollutant selected")

        min_val, max_val = fetch_threshold_range(pollutant)
        if min_val >= max_val:
            raise ValueError("Invalid threshold range")

        # CASE 1: range < 1  → precisione 2 decimali
        if max_val < 1:
            min_v = round(min_val, 1)
            max_v = round(max_val, 1)
            step = 0.01
            value = round((min_v + max_v) / 2, 2)
            delta = (max_v - min_v) / 4
            marks = {
                round(min_v + i * delta, 1): str(round(min_v + i * delta, 1))
                for i in range(5)
            }
            return min_v, max_v, step, value, marks

        # CASE 2: 1 ≤ range ≤ 10 → precisione 1 decimale 
        elif max_val <= 10:
            min_v = round(min_val, 1)
            max_v = round(max_val, 1)
            step = 0.1
            value = round((min_v + max_v) / 2, 1)
            delta = (max_v - min_v) / 4
            marks = {
                round(min_v + i * delta, 1): str(round(min_v + i * delta, 1))
                for i in range(5)
            }
            return min_v, max_v, step, value, marks

        # CASE 3: range > 10 → slider intero 
        else:
            min_v = int(math.floor(min_val))
            max_v = int(math.ceil(max_val))
            step = 1
            value = int(round((min_v + max_v) / 2))
            delta = (max_v - min_v) / 4
            marks = {
                int(round(min_v + i * delta)): str(int(round(min_v + i * delta)))
                for i in range(5)
            }
            return min_v, max_v, step, value, marks

    except Exception as e:
        return 0, 80, 1, 50, {i: str(i) for i in range(0, 201, 20)}


if __name__ == '__main__':
    print("Starting Dashboard...")
    app.run(port=8089, debug=True)




Starting Dashboard...
